# An Simple Linear Learner Example for Tip Prediction

_**Predicting a tip from the bill amount**_

1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
  1. [Permissions and environment variables](#Permissions-and-environment-variables)
  2. [Data ingestion](#Data-ingestion)
  3. [Data inspection](#Data-inspection)
  4. [Data conversion](#Data-conversion)
3. [Training the linear model](#Training-the-linear-model)
4. [Set up hosting for the model](#Set-up-hosting-for-the-model)
5. [Validate the model for use](#Validate-the-model-for-use)


## Introduction

In this lab, we will predict the best estimate for the tip given the amount of the bill in a restaurant. This is the simplest possible example but it will touch on many important aspects of machine learning, as well as using SageMaker. The bill and tip data is found here: https://s3.amazonaws.com/elephantscale-public/data/tips/bill-and-tips-sample.csv.

To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on.

## Prequisites and Preprocessing

### Permissions and environment variables

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
## TODO : Change this bucket name to match your environment
bucket = "elephantscale-sagemaker"
output_location = 's3://elephantscale-sagemaker/output'

In [ ]:
prefix = 'sagemaker/lr-tips'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

### Data ingestion

Next, we read the dataset from an online URL into memory, for preprocessing prior to training.

In [ ]:
%%time
import numpy, urllib.request, json

# Load the dataset
urllib.request.urlretrieve("https://s3.amazonaws.com/elephantscale-public/data/tips/bill-and-tips-sample.csv", 
                           "bill-and-tips-sample.csv")

### Data inspection

Let's read and inspect the data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas 

#df = pandas.read_table('./bill-and-tips-sample.csv', delim_whitespace=True, names=('bill', 'tip'))
bill_and_tip = pandas.read_table('./bill-and-tips-sample.csv', sep = ',')
print(bill_and_tip)

### Data plotting

Let us plot the data, to confirm our idea of applying linear regression to it.

In [ ]:
plt.scatter(bill_and_tip.bill, bill_and_tip.tip)
plt.ylabel('tip')
plt.xlabel('bill')
plt.show()

In [ ]:
print(bill_and_tip.tip)

## Upload training data
The data is tiny and is in a very simple, CSV format. Yet, we'll need to upload it to S3, so that Amazon SageMaker training can use it.

In [ ]:
import io
import numpy as np
import sagemaker.amazon.common as smac

vectors = np.array(list(map (lambda x : [x], bill_and_tip.bill.tolist()))).astype('float32')
labels = np.array(bill_and_tip.tip).astype('float32')
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)

In [ ]:
import boto3
import os

key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

Let's also setup an output S3 location for the model artifact that will be output as the result of training with the algorithm.

## Training the linear model

Once we have the data preprocessed and available in the correct format for training, the next step is to actually train the model using the data. Since this data is relatively small, it isn't meant to show off the performance of the Linear Learner training algorithm, although we have tested it on multi-terabyte datasets.

Again, we'll use the Amazon SageMaker Python SDK to kick off training, and monitor status until it is completed.  In this example that takes between 7 and 11 minutes.  Despite the dataset being small, provisioning hardware and loading the algorithm container take time upfront.

First, let's specify our containers.  Since we want this notebook to run in all 4 of Amazon SageMaker's regions, we'll create a small lookup.  More details on algorithm containers can be found in [AWS documentation](https://docs-aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html).

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

Next we'll kick off the base estimator, making sure to pass in the necessary hyperparameters.  Notice:
- `feature_dim` is set to 1, for just one predictor, the bill amount.
- `predictor_type` is set to 'regressor' since we are trying to predict a continous variable output (tip).
- `mini_batch_size` is set to 10.  This value can be tuned for relatively minor improvements in fit and speed, but selecting a reasonable value relative to the dataset is appropriate in most cases.

In [ ]:
import boto3
import sagemaker



sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=1,
                           predictor_type='regressor',
                           mini_batch_size=10)

linear.fit({'train': s3_train_data})

## Set up hosting for the model
Now that we've trained our model, we can deploy it behind an Amazon SageMaker real-time hosted endpoint.  This will allow out to make predictions (or inference) from the model dynamically.

In [ ]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

## Validate the model for use
Finally, we can now validate the model for use.  We can pass HTTP POST requests to the endpoint to get back predictions.  To make this easier, we'll again use the Amazon SageMaker Python SDK and specify how to serialize requests and deserialize responses that are specific to the algorithm.

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

Now let's try getting a prediction for a single record.

In [ ]:
print("Tip amount for $100 restaurant bill")
result = linear_predictor.predict([100.0])
print(result)

### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(linear_predictor.endpoint)